### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 28 мая 2022, 08:30   
**Штраф за опоздание:** по 1 баллу за 24 часа задержки.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0221, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [55]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [195]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        if self.criterion == 'gini':
            self.func = self.__gini
        if self.criterion == 'entropy':
            self.func = self.__entropy
        if self.criterion == 'misclass':
            self.func = self.__misclass
        self.summary_gain = {}
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __entropy(self, left_prob, l_p, right_prob, r_p, split_ind):
        def entr(x):
            x = np.asarray(x)
            x[x != 0] = -x[x != 0] * np.log2(x[x!=0])
            return x
        return left_prob*(np.sum(entr(l_p),axis = 0)) + \
               right_prob*(np.sum(entr(r_p),axis = 0)), split_ind

    def __gini(self, left_prob, l_p, right_prob, r_p, split_ind):
        return left_prob*(1 - np.sum(np.square(l_p), axis = 0)) + \
               right_prob*(1 - np.sum(np.square(r_p), axis = 0)), split_ind

    def __misclass(self, left_prob, l_p, right_prob, r_p, split_ind):
        return left_prob*(1 - np.max(l_p, axis = 0)) + \
               right_prob*(1 - np.max(r_p, axis = 0)), split_ind

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] <= threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def _find_border(self, x):
        """
        [0 0 0 1 1 1 1] -> ([0, 1], [3, 4])
        """
        x = np.append(np.asarray(x), -1)
        border = np.where(x[1:] != x[:-1])[0]
        return np.append(border[0] + 1, border[1:] - border[:-1]), x[border]

    def _find_probs(self, counts, borders):
        self.num_cls = np.max(borders) + 1
        history = np.zeros((self.num_cls, counts.size))
        history[borders, np.arange(counts.size)] = counts
        left_history = np.cumsum(history, axis = 1)
        right_history = left_history[:, -1].reshape(-1, 1) - left_history
        left_sum = np.sum(left_history, axis = 0)
        right_sum = np.sum(right_history, axis = 0)
        total_sum = np.sum(left_history[:, -1])
        pl = left_history / left_sum
        pr = np.nan_to_num(right_history / right_sum, 0)
        left_split_pr = left_sum / total_sum
        return left_split_pr, pl, 1 - left_split_pr, pr, left_sum

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        ind = np.argsort(x)
        x, y = x[ind], y[ind]
        probs = self._find_probs(*self._find_border(y))
        loss, split_ind =  self.func(*probs)
        target_ind = np.argmin(loss)
        return np.array([loss[target_ind], x[np.int64(split_ind[target_ind]) - 1]])

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        loss_data = np.array([self.__find_threshold(x[:, i], y) for i in range(x[1].size)])
        split_dim = np.argmin(loss_data[:,0])
        threshold = loss_data[split_dim][1]
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, split_dim, threshold)
        #print(y_left, y_right)
        if x_right.size != 0 and depth < self.max_depth:
            self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, split_dim, threshold)
            self.summary_gain[split_dim] = self.summary_gain.get(split_dim, 0) + loss_data[split_dim][0]
            if np.all(y_left == y_left[0]):
                self.tree[node_id * 2 + 1] = (self.__class__.LEAF_TYPE, y_left[0], 1)
            else:
                self.__fit_node(x_left, y_left, node_id * 2 + 1, depth + 1)
            if np.all(y_right == y_right[0]):
                self.tree[node_id * 2 + 2] = (self.__class__.LEAF_TYPE, y_right[0], 1)
            else:
                self.__fit_node(x_right, y_right, node_id * 2 + 2, depth + 1)
        else:
            cur = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, np.argmax(cur), cur[np.argmax(cur)]/np.sum(cur))
            #print(np.argmax(cur), cur[np.argmax(cur)]/np.sum(cur))
        
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] <= threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance(self):
        """
        Возвращает важность признаков
        """
        dims, vals = zip(*sorted(self.summary_gain.items(), key=lambda x: x[1]))
        print(dims, vals)
        return dims

In [196]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=3)
my_clf.fit(np.arange(10).reshape(-1,1), np.array([2,2,2,1,1,1,2,2,1,1]))
my_clf.predict([[7]])
my_clf.get_feature_importance()

(0,) (0.5714285714285714,)


(0,)

In [230]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=5, criterion='misclass')
clf = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
my_clf.fit(X_train, y_train), clf.fit(X_train, y_train)
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))
print(my_clf.get_feature_importance())

0.7777777777777778
0.7777777777777778
(1, 0, 9, 6, 2, 5, 11, 12) (0.0, 0.0, 0.0, 0.03030303030303028, 0.032258064516128976, 0.050847457627118633, 0.06930693069306934, 0.3)
(1, 0, 9, 6, 2, 5, 11, 12)


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [6]:
%time clf.fit(X_train, y_train)

CPU times: user 2.23 ms, sys: 1.34 ms, total: 3.57 ms
Wall time: 2.2 ms


DecisionTreeClassifier(max_depth=5)

In [7]:
%time my_clf.fit(X_train, y_train)

CPU times: user 16.3 ms, sys: 1.56 ms, total: 17.9 ms
Wall time: 16.5 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [102]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
print(df.shape, 'match' in df.columns)
df.head()

(8378, 195) True


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [103]:
for i in df.columns:
    if df[i].isnull().sum() > 1 / 10 * df.shape[0]:
        df.drop(i, axis = 1, inplace=True)
df.shape

(8378, 87)

In [104]:
for num, row in df.iterrows():
    if row.isnull().sum() > 0:
        df.drop(num, axis = 0, inplace=True)
df.shape

(6193, 87)

In [105]:
for i in df.columns:
    if isinstance(df.loc[30, i], str):
        df.drop(i, axis = 1, inplace=True)
df.shape

(6193, 84)

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [258]:
y = df.loc[:, 'match'].values
X = df.drop('match', axis = 1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [259]:
my_clf = MyDecisionTreeClassifier(max_depth=10, criterion='gini')
clf = DecisionTreeClassifier(max_depth=10, criterion='entropy')
my_clf.fit(X_train, y_train), clf.fit(X_train, y_train)
f1_score(my_clf.predict(X_test), y_test), f1_score(clf.predict(X_test), y_test)

(1.0, 1.0)

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [261]:
my_clf.get_feature_importance()

(74, 21) (0.0, 0.2053167956583031)


(74, 21)

In [260]:
clf.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.36672085, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.63327

In [264]:
df.columns[74], df.columns[21]

('amb3_1', 'pf_o_sha')

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

